# *Jones* Model Examples for Radar Polarimetry Review

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import cm

# Impdar is a dependency here: download from https://github.com/dlilien/ImpDAR
from impdar.lib.ApresData.load_quadpol import load_quadpol_fujita
from impdar.lib.ApresData._QuadPolProcessing import power_anomaly, lowpass

from effmed.lib.matrix_model import effective_medium
from effmed.lib.supplemental import dB, fresnel

%matplotlib widget

### Example 1: Uniform Anisotropic Transmission (no rotation)

In [ ]:
em = effective_medium()

fc = 300e6
em.system_setup(fc)

em.epsr=3.15
chis = [0.,0.1,0.9]
em.ice_properties(T=253.,epsr=3.15,chi=chis)

dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(chis[1]-chis[0])/(2.*np.sqrt(em.epsr)))
H = 2.*np.pi/dphidz
layer_dz = 1.
zs = np.arange(.1,H+8.*layer_dz,layer_dz)
dzs = layer_dz*np.ones(len(zs))
chi1 = 0.0*np.ones(len(zs))
chi2 = 0.1*np.ones(len(zs))
chi3 = 0.9*np.ones(len(zs))
chis = np.transpose([chi1,chi2,chi3])
thetas = np.zeros(len(zs))
psis = np.zeros(len(zs))

em.solve(zs,dzs,thetas,psis,chis)

dat = load_quadpol_fujita(em)
dat.rotational_transform(n_thetas=401)
Θs,Ds = np.meshgrid(dat.thetas,dat.range)
dat.find_cpe(5e7,rad_start=.1*np.pi,rad_end=.6*np.pi)
dat.cpe[dat.cpe==.1*np.pi] = .5*np.pi

dat.coherence2d(delta_theta=.008,delta_range=1.1)
dat.phase_gradient2d()

Θs,Ds = np.meshgrid(dat.thetas,dat.range)

In [ ]:
dphi_dz = dat.dphi_dz.copy()
dphi_dz[dphi_dz<-0.05] = -0.05
dphi_dz[dphi_dz>0.05] = 0.05

c_hhvv = np.angle(dat.HH)-np.angle(dat.VV)
c_hhvv[c_hhvv<-np.pi] += 2.*np.pi
c_hhvv[c_hhvv>np.pi] -= 2.*np.pi

E0x = abs(dat.HH)
E0y = abs(dat.HV)
ϕHH = np.angle(dat.HH)
ϕHV = np.angle(dat.HV)
ϕ = ϕHH-ϕHV
ϕ[ϕ<-np.pi] += 2.*np.pi
ϕ[ϕ>np.pi] -= 2.*np.pi
γ = np.arctan(E0y/E0x)

In [ ]:
plt.rcParams['axes.linewidth'] = 1.0

plt.figure(figsize=(10,6))
gs = GridSpec(4,8)

ax1 = plt.subplot(gs[:2,:2])
plt.tick_params(labelleft=True,labelbottom=False,bottom=False,labeltop=True,top=True)
plt.contourf(Θs,Ds,np.real(dB(dat.HH**2.)),levels=np.linspace(-10,0,100),cmap='Greys_r',extend='min',rasterized=True)
cbar = plt.colorbar(orientation='horizontal',location='bottom',label='Co-Polarized Power (dB)')
cbar.set_ticks(np.linspace(-10,0,3))
plt.xlabel('Azimuth ($\psi$)')
plt.ylabel('Depth')
ax1.xaxis.set_label_position('top') 
ax1.set_xticks([0,np.pi/2.,np.pi])
ax1.set_xticklabels(['0','π/2','π'])
#plt.ylim(max(dat.range),min(dat.range))
plt.ylim(H,0)
ax1.set_yticks([min(zs),H/2,H])
ax1.set_yticklabels(['0','L/2','L'])
ax1.text(0.075,.8,'a',transform=ax1.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))

ax2 = plt.subplot(gs[:2,2:4])
plt.tick_params(labelleft=False,labelbottom=False,bottom=False,labeltop=True,top=True)
ax2.set_xticks([0,np.pi/2.,np.pi])
plt.contourf(Θs,Ds,np.real(dB(dat.HV**2.)),levels=np.linspace(-10,0,100),cmap='Greys_r',extend='min',rasterized=True)
cbar = plt.colorbar(orientation='horizontal',location='bottom',label='Cross-Polarized Power (dB)')
cbar.set_ticks(np.linspace(-10,0,3))
plt.xlabel('Azimuth ($\psi$)')
ax2.xaxis.set_label_position('top') 
ax2.set_xticks([0,np.pi/2.,np.pi])
ax2.set_xticklabels(['0','π/2','π'])
#plt.ylim(max(dat.range),min(dat.range))
plt.ylim(H,0)
ax2.set_yticks([min(zs),H/2,H])
ax2.text(0.075,.8,'b',transform=ax2.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))

ax3 = plt.subplot(gs[:2,4:6])
plt.tick_params(labelleft=False,labelbottom=False,bottom=False,labeltop=True,top=True)
ax3.set_xticks([0,np.pi/2.,np.pi])
plt.contourf(Θs,Ds,c_hhvv,aspect='auto',cmap='twilight_shifted',levels=np.linspace(-np.pi,np.pi,100),rasterized=True)
cbar = plt.colorbar(orientation='horizontal',location='bottom',label='Co-Polarized Phase ($ϕ_{HHVV}$)')
cbar.set_ticks(np.linspace(-np.pi,np.pi,3))
cbar.set_ticklabels(['-π','0','π'])
plt.xlabel('Azimuth ($\psi$)')
ax3.xaxis.set_label_position('top') 
ax3.set_xticks([0,np.pi/2.,np.pi])
ax3.set_xticklabels(['0','π/2','π'])
#plt.ylim(max(dat.range),min(dat.range))
plt.ylim(H,0)
ax3.set_yticks([min(zs),H/2,H])
ax3.text(0.075,.8,'c',transform=ax3.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))

ax4 = plt.subplot(gs[:2,6:])
plt.tick_params(labelleft=False,labelbottom=False,bottom=False,labeltop=True,top=True)
ax4.set_xticks([0,np.pi/2.,np.pi])
plt.contourf(Θs,Ds,dphi_dz,aspect='auto',cmap='RdYlBu_r',levels=np.linspace(-0.05,0.05,100),rasterized=True)
cbar = plt.colorbar(orientation='horizontal',location='bottom',label='Phase Gradient ($\partial\phi/\partial z$)')
cbar.set_ticks([-0.05,0,0.05])
cbar.set_ticklabels(['-0.05','0','0.05'])
plt.xlabel('Azimuth ($\psi$)')
ax4.xaxis.set_label_position('top') 
ax4.set_xticks([0,np.pi/2.,np.pi])
ax4.set_xticklabels(['0','π/2','π'])
#plt.ylim(max(dat.range),min(dat.range))
plt.ylim(H,0)
ax4.set_yticks([min(zs),H/2,H])
ax4.text(0.075,.8,'d',transform=ax4.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))

thidxs = [1,15,100,150]
didxs = [0,65,130,195,260,325,390,455,520]
labels=['g','h','i','j']
for i,thidx in enumerate(thidxs):
    ax5i = plt.subplot(gs[2:,i+4])
    ax5i.tick_params(labelleft=False,left=False,labelbottom=False,bottom=False)
    plt.ylim(-1,1)
    plt.xlim(-1,1)
    for j,didx in enumerate(didxs):
        lHV = abs(dat.HV[didx,thidx])*np.sin(np.linspace(-np.pi,np.pi,50)-ϕHV[didx,thidx])
        lHH = abs(dat.HH[didx,thidx])*np.sin(np.linspace(-np.pi,np.pi,50)-ϕHH[didx,thidx])
        plt.plot(lHH,lHV-.5*j,alpha=1.,c=cm.twilight_shifted((ϕ[didx,thidx]+np.pi)/(2*np.pi)))
    plt.axis('equal')
    ax5i.text(0.05,.95,labels[i],transform=ax5i.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))
    for spine in ax5i.spines:
        ax5i.spines[spine].set_visible(False)
    
ax6 = plt.subplot(gs[2:,:2])
plt.tick_params(labelleft=True,labelbottom=False,bottom=False,top=True)
ax6.set_xticks([0,np.pi/2.,np.pi])
plt.contourf(Θs,Ds,γ,cmap='Greys_r',levels=np.linspace(0,np.pi/2.,100),rasterized=True)
cbar = plt.colorbar(orientation='horizontal',label='Ellipse Rotation ($\gamma$)')
cbar.set_ticks(np.linspace(0,np.pi/2.,3))
cbar.set_ticklabels(['0','π/4','π/2'])
#plt.ylim(max(dat.range),min(dat.range))
plt.ylabel('Depth')
plt.ylim(H,0)
ax6.set_yticks([min(zs),H/2,H])
ax6.set_yticklabels(['0','L/2','L'])
ax6.text(0.075,.8,'e',transform=ax6.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))

ax7 = plt.subplot(gs[2:,2:4])
plt.tick_params(labelleft=False,labelbottom=False,bottom=False,top=True)
ax7.set_xticks([0,np.pi/2.,np.pi])
plt.contourf(Θs,Ds,ϕ,cmap='twilight_shifted',levels=np.linspace(-np.pi,np.pi,100),rasterized=True)
cbar = plt.colorbar(orientation='horizontal',label='Ellipse Phase Delay ($\phi_{ellipse}$)')
cbar.set_ticks(np.linspace(-np.pi,np.pi,3))
cbar.set_ticklabels(['-π','0','π'])
for i,th in enumerate(thidxs):
    plt.arrow(dat.thetas[th],-60,0,30,color='k',clip_on=False,head_width=.05,head_length=20)
    if i == 0:
        plt.text(dat.thetas[th]-.02,-70,labels[i],ha='center',size=10)
    elif i == 1:
        plt.text(dat.thetas[th]+.02,-70,labels[i],ha='center',size=10)
    else:
        plt.text(dat.thetas[th],-70,labels[i],ha='center',size=10)
plt.xlim(min(dat.thetas),max(dat.thetas))
#plt.ylim(max(dat.range),min(dat.range))
plt.ylim(H,0)
ax7.set_yticks([min(zs),H/2,H])
ax7.text(0.075,.8,'f',transform=ax7.transAxes,fontweight='bold',fontsize=14,
       bbox=dict(facecolor='w', edgecolor='k', pad=5.0, linewidth=2.))

plt.tight_layout()

plt.savefig('Appendix_Polarimetry_full.pdf',dpi=600)

### Example 2: Anisotropic Reflectivity

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = np.array([.333,.334,.333])
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(.1)/(2.*np.sqrt(em.epsr)))
H = 2.*np.pi/dphidz
zs = np.arange(10,H)
thetas=0.
psis = 0.
gammas = np.array([1.,.3])
em.solve(zs,H,thetas,psis,chis,gammas=gammas)

dat1 = load_quadpol_fujita(em)
dat1.rotational_transform(n_thetas=201)
dat1.find_cpe()
dat1.coherence2d(.15,2.)

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [0.,.1,0.9]
em.ice_properties(T=253.,epsr=3.15,chi=chis)

dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(chis[1]-chis[0])/(2.*np.sqrt(em.epsr)))
H = 2.*np.pi/dphidz
zs = np.arange(10,H)
thetas=0.
psis = 0.
gammas = np.array([1.,.3])
em.solve(zs,H,thetas,psis,chis,gammas=gammas)

dat2 = load_quadpol_fujita(em)
dat2.rotational_transform(n_thetas=201)
dat2.find_cpe()
dat2.coherence2d(.15,2.)

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [0.,.1,0.9]
em.ice_properties(T=253.,epsr=3.15,chi=chis)

dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(chis[1]-chis[0])/(2.*np.sqrt(em.epsr)))
H = 2.*np.pi/dphidz
#H = 1000.
zs = np.arange(10,H)
thetas=0.
psis = 0.
gammas = np.array([1.,.3])
em.solve(zs,H,thetas,psis,chis,gammas=gammas,psi_gammas=0.15*np.pi)
#em.solve(zs,[H-.01,.01],[0,0],[0,.1*np.pi],[chis,chis],gammas=gammas)

dat3 = load_quadpol_fujita(em)
dat3.rotational_transform(n_thetas=201)
dat3.find_cpe()
dat3.coherence2d(.15,2.)

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(8,4))
gs = GridSpec(3,4)

ax0 = plt.subplot(gs[1,3])
plt.tick_params(labelleft=False,left=False,right=True)
ψs = np.linspace(0.00001,np.pi,1000)
rs = 10.*np.log10(1/(np.tan(ψs/2)**(2.)))
plt.plot(rs,ψs,'k')
plt.xlim(-40,40)
plt.ylim(0,np.pi)
plt.yticks([0,np.pi/2.,np.pi])
ax0.set_yticklabels(['0','π/2','π'])
ax0.text(0.875,.8,'j',transform=ax0.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
       bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
plt.ylabel('Azimuthal Distance ($\psi_{AD}$)')
plt.xlabel('Reflection Ratio (dB)')
ax0.yaxis.set_label_position('right') 

dats = [dat1,dat2,dat3]
labels = ['a','b','c','d','e','f','g','h','i']

for i,dat in enumerate(dats):
    Θs,Ds = np.meshgrid(dat.thetas,dat.range)

    ax1 = plt.subplot(gs[i,0],facecolor='k')
    im1 = plt.pcolormesh(Θs,Ds,np.real(dB(dat.HH**2.)),cmap='Greys_r',vmin=-20,vmax=0.,rasterized=True)
    ax1.text(0.1,.8,labels[i*3],transform=ax1.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
           bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
    plt.ylim(H,10)
    plt.yticks([10,(H-10)/2.,H])
    plt.xticks([0,np.pi/2.,np.pi])
    if i != 2:
        ax1.set_xticklabels(['','',''])
        plt.tick_params(labelleft=False)
    else:
        ax1.set_xticklabels(['0','π/2','π'])
        ax1.set_yticklabels(['0','L/2','L'])
        ax1.set_xlabel('Azimuth ($\psi$)')
        ax1.set_ylabel('Depth')
        #ax3.yaxis.set_label_position('right') 
    
    ax2 = plt.subplot(gs[i,1],facecolor='k')
    plt.tick_params(labelleft=False)
    im2 = plt.pcolormesh(Θs,Ds,np.real(dB(dat.HV**2.)),cmap='Greys_r',vmin=-20,vmax=0.,rasterized=True)
    ax2.text(0.1,.8,labels[i*3+1],transform=ax2.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
           bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
    plt.ylim(H,10)
    plt.yticks([10,(H-10)/2.,H])
    plt.xticks([0,np.pi/2.,np.pi])
    ax2.set_xticklabels(['','',''])

    ax3 = plt.subplot(gs[i,2])
    plt.tick_params(labelleft=False)
    im3 = plt.pcolormesh(Θs,Ds,np.angle(dat.chhvv),cmap='twilight_shifted',vmin=-np.pi,vmax=np.pi,rasterized=True)
    ax3.text(0.1,.8,labels[i*3+2],transform=ax3.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
           bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
    plt.ylim(H,10)
    plt.yticks([10,(H-10)/2.,H])
    plt.xticks([0,np.pi/2.,np.pi])
    ax3.set_xticklabels(['','',''])
    if i == 2:
        plt.xlabel('$\psi_\Gamma$')

# -----------------------------------------------------------------------------------
# Colorbars
# -----------------------------------------------------------------------------------

fig.subplots_adjust(top=0.8)
cbar1_ax = fig.add_axes([.14, 0.825, .15, 0.02])
cbar2_ax = fig.add_axes([.34, 0.825, .15, 0.02])
cbar3_ax = fig.add_axes([.54, 0.825, .15, 0.02])
cbar1 = fig.colorbar(im1,cax=cbar1_ax,label='Co-Pol Power (dB)',extend='min',orientation='horizontal',ticks=[-20,-10,0])
cbar2 = fig.colorbar(im2,cax=cbar2_ax,label='Cross-Pol Power (dB)',extend='min',orientation='horizontal',ticks=[-20,-10,0])
cbar3 = fig.colorbar(im3,cax=cbar3_ax,label='Phase Angle ($\phi_{HHVV}$)',orientation='horizontal',ticks=[-np.pi,0,np.pi])
cbar3.ax.set_xticklabels(['-π','0','π'])

for cbar in [cbar1,cbar2,cbar3]:
    cbar.ax.xaxis.tick_top()
    cbar.ax.xaxis.set_label_position('top') 

plt.savefig('eff_med_scattering.pdf',dpi=300)

### Rotations

In [ ]:
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.15,.35,0.5]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(chis[1]-chis[0])/(2.*np.sqrt(em.epsr)))
H1 = 4.*np.pi/dphidz
layer_dz = 2.
zs = np.arange(1,H1,layer_dz)
thetas = np.zeros(len(zs))
psis1 = .01+np.zeros_like(zs)
psis1[zs>200] = .6
dzs = layer_dz*np.ones(len(zs))
chis = np.tile(chis,(len(zs),1))
em.solve(zs,dzs,thetas,psis1,chis)

dat1 = load_quadpol_fujita(em)
dat1.rotational_transform(n_thetas=201)
dat1.find_cpe(Wn=3e7,rad_start=0.5*np.pi,rad_end=np.pi)
dat1.coherence2d(.15,layer_dz+.1)

In [ ]:
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.225,.275,0.5]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(chis[1]-chis[0])/(2.*np.sqrt(em.epsr)))
H2 = 4.*np.pi/dphidz
layer_dz = 10.
zs = np.arange(1,H2,layer_dz)
thetas = np.zeros(len(zs))
psis2 = np.linspace(0,np.pi/4.,len(zs))
dzs = layer_dz*np.ones(len(zs))
chis = np.tile(chis,(len(zs),1))

em.solve(zs,dzs,thetas,psis2,chis)

dat2 = load_quadpol_fujita(em)
dat2.rotational_transform(n_thetas=401)
dat2.find_cpe(Wn=3e6,rad_start=0.5*np.pi,rad_end=np.pi)
dat2.coherence2d(.15,layer_dz+.1)

Θs,Ds = np.meshgrid(dat2.thetas,dat2.range)

In [ ]:
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.15,.35,0.5]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(chis[1]-chis[0])/(2.*np.sqrt(em.epsr)))
H3 = 4.*np.pi/dphidz
layer_dz = 2.
zs = np.arange(1,H3,layer_dz)
thetas = np.zeros(len(zs))
psis3 = .4*np.tanh(.01*(zs-250))+.3
dzs = layer_dz*np.ones(len(zs))
chis = np.tile(chis,(len(zs),1))
em.solve(zs,dzs,thetas,psis3,chis)

dat3 = load_quadpol_fujita(em)
dat3.rotational_transform(n_thetas=201)
dat3.find_cpe(Wn=3e7,rad_start=0.4*np.pi,rad_end=0.9*np.pi)
dat3.coherence2d(.15,layer_dz+.1)

In [ ]:
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.15,.35,0.5]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

dphidz = (4.*np.pi*em.fc/em.c)*(em.depsr*(chis[1]-chis[0])/(2.*np.sqrt(em.epsr)))
H4 = 4.*np.pi/dphidz
layer_dz = 2.
zs = np.arange(1,H4,layer_dz)
thetas = np.zeros(len(zs))
psis4 = .01+np.zeros_like(zs)
psis4[np.logical_and(zs>120,zs<200)] = .4
dzs = layer_dz*np.ones(len(zs))
chis = np.tile(chis,(len(zs),1))
em.solve(zs,dzs,thetas,psis4,chis)

dat4 = load_quadpol_fujita(em)
dat4.rotational_transform(n_thetas=201)
dat4.find_cpe(Wn=3e7,rad_start=0.4*np.pi,rad_end=0.9*np.pi)
dat4.coherence2d(.15,layer_dz+.1)

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(8,4))

dats = [dat1,dat2,dat3,dat4]
psis = [psis1,psis2,psis3,psis4]
Hs = [H1,H2,H3,H4]
labels = ['a','b','c','d','e','f','g','h']

cred = 'darkmagenta'
cgreen = 'forestgreen'

for i,dat in enumerate(dats):
    Θs,Ds = np.meshgrid(dat.thetas,dat.range)
    H = Hs[i]

    ax1 = plt.subplot(2,4,i*2+1,facecolor='k')
    im1 = plt.pcolormesh(Θs,Ds,np.real(dB(dat.HV**2.)),cmap='Greys_r',vmin=-10,vmax=0.,rasterized=True)
    ax1.text(0.1,.87,labels[i*2],transform=ax1.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
           bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
    plt.plot(dat.cpe,Ds,'.',c=cred,ms=.5,rasterized=True)
    plt.plot(psis[i]+np.pi/2.,Ds,'.',c=cgreen,ms=.5,rasterized=True)
    plt.ylim(H,0)
    plt.yticks([0,H/4,H/2,3*H/4,H])
    plt.xticks([0,np.pi/2.,np.pi])
    if i == 2:
        ax1.set_ylabel('Depth')
        ax1.set_xticklabels(['0','π/2','π'])
        ax1.set_xlabel('Azimuth (ψ)')
        ax1.set_yticklabels(['0','','L','','2L'])
    else:
        ax1.set_xticklabels(['','',''])
        ax1.tick_params(labelleft=False)

    ax2 = plt.subplot(2,4,i*2+2)
    im2 = plt.pcolormesh(Θs,Ds,np.angle(dat.chhvv),cmap='twilight_shifted',vmin=-np.pi,vmax=np.pi,rasterized=True)
    ax2.text(0.1,.87,labels[i*2+1],transform=ax2.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
           bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
    plt.ylim(H,0)
    plt.xticks([0,np.pi/2.,np.pi])
    plt.tick_params(labelleft=False)
    ax2.set_xticklabels(['','',''])
    
plt.tight_layout()
    
# -----------------------------------------------------------------------------------
# Colorbars
# -----------------------------------------------------------------------------------

fig.subplots_adjust(top=0.8)
cbar1_ax = fig.add_axes([0.135, 0.825, 0.3, 0.02])
cbar2_ax = fig.add_axes([0.61, 0.825, 0.3, 0.02])
cbar1 = fig.colorbar(im1,cax=cbar1_ax,label='Cross-Pol Power (dB)',extend='min',orientation='horizontal',ticks=[-10,-5,0])
cbar2 = fig.colorbar(im2,cax=cbar2_ax,label='Phase Delay ($\phi_{HHVV}$)',orientation='horizontal',ticks=[-np.pi,0,np.pi])
cbar1.ax.xaxis.tick_top()
cbar1.ax.xaxis.set_label_position('top') 
cbar2.ax.set_xticklabels(['-π','0','π'])
cbar2.ax.xaxis.tick_top()
cbar2.ax.xaxis.set_label_position('top') 

plt.savefig('eff_med_rotation.pdf',dpi=600)

### Power Loss and Oblique Propagation

In [ ]:
θ = 0.*np.pi/180.
ψ = 45.*np.pi/180.
fc = 3e8
d = 1e3
dχ = .1
χ = np.array([(.3-dχ)/2.,(+dχ)/2.,0.7])

em0 = effective_medium()
em0.system_setup(fc=fc)
em0.epsr = 3.15
em0.ice_properties(idctx='vertical-girdle',chi=χ, epsr=em0.epsr, T=253)

dphidz = (4.*np.pi*em0.fc/em0.c)*(em0.depsr*dχ/(2.*np.sqrt(3.15)))
H = 2.*2.*np.pi/dphidz

zs = np.arange(.1,H)
em0.solve(zs,d,θ,ψ,χ)

In [ ]:
fig = plt.figure(figsize=(10,4))
gs = GridSpec(2,5)

# Plot 1-d power loss along a single azimuth
ax0 = plt.subplot(gs[:,0])
plt.plot(np.real(dB(em0.shh**2.)),em0.range,'k')
plt.xlim(-50,0.5)
plt.ylim(H,0)
plt.ylabel('Depth')
plt.xlabel('Co-Pol Power (dB)')
ax0.text(0.12,.93,'a',transform=ax0.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
       bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
plt.yticks([0,H/2.,H])
ax0.set_yticklabels(['0','L','2L'])


fcs = [1e8,5e8,1e9,2e9]
θs = [10,10,10,10]
labels = ['b','c','d','e']
for pi in range(4):

    em = effective_medium()

    θ = [θs[pi]*np.pi/180.]
    ψ = [90.*np.pi/180.]
    θw,ψw = np.meshgrid(np.linspace(0,40,200)*np.pi/180.,np.linspace(0,90,200)*np.pi/180.)

    fc = fcs[pi]
    d = 1e3
    dz = 1e3
    χ = np.array([[0,0,1]])

    AMP = np.empty_like(θw)
    for i in range(len(θw)):
        for j in range(len(θw[0])):
            em.system_setup(fc=fc, psi_w=ψw[i,j], theta_w=θw[i,j])
            em.ice_properties(idctx='single-pole',theta=θ[0],psi=ψ[0],chi=χ[0])
            em.single_depth_solve(d,[dz],θ,ψ,χ,gamma=[1., 1.],idctx='single-pole')
            AMP[i,j] = np.real(10.*np.log10(em.S[0,0]**2.))
            
    ax = plt.subplot(gs[0,pi+1])
    ax.tick_params(labelleft=False,labelbottom=False)
    plt.pcolormesh(ψw,θw,AMP,cmap='Greys_r', vmin=-5,vmax=0)
    ax.set_xticks([0,np.pi/4.,np.pi/2.])
    ax.yaxis.tick_right()
    ax.text(0.1,.87,labels[pi],transform=ax.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
           bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))

    
fcs = [1e8,5e8,1e9,2e9]
dχs = [.1,.1,.1,.1]
labels = ['f','g','h','i']
for pi in range(4):
    em = effective_medium()

    θ = 0.*np.pi/180.
    ψ = 90.*np.pi/180.
    θw,ψw = np.meshgrid(np.linspace(0,40,200)*np.pi/180.,np.linspace(0,90,200)*np.pi/180.)

    fc = fcs[pi]
    d = 1e3
    dz = 1e3
    χ = np.array([(.3+dχs[pi])/2.,(.3-dχs[pi])/2.,0.7])

    AMP = np.empty_like(θw)
    for i in range(len(θw)):
        for j in range(len(θw[0])):
            em.system_setup(fc=fc, psi_w=ψw[i,j], theta_w=θw[i,j])
            em.ice_properties(idctx='vertical-girdle',chi=χ, epsr=3.152, T=320)
            em.single_depth_solve(d,dz,θ,ψ,χ,gamma=[1., 1.],idctx='vertical-girdle')
            AMP[i,j] = np.real(10.*np.log10(em.S[0,0]**2.))
            
    ax = plt.subplot(gs[1,pi+1])
    im = plt.pcolormesh(ψw,θw,AMP,cmap='Greys_r', vmin=-5,vmax=0)
    ax.set_yticks([0,np.pi/16.,np.pi/8.,3*np.pi/16.])
    ax.set_xticks([0,np.pi/4.,np.pi/2.])
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right")
    if pi == 3:
        ax.set_ylabel(r'Polar Angle ($\theta$)')
        ax.set_xlabel('Azimuth Angle ($\psi$)')
        ax.set_yticklabels(['0','π/16','π/8','3π/16'])
        ax.set_xticklabels(['0','π/4','π/2'])
    else:
        ax.tick_params(labelleft=False,labelbottom=False,labelright=False)
    ax.text(0.1,.87,labels[pi],transform=ax.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
           bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
    
fig.subplots_adjust(top=0.8)
cbar_ax = fig.add_axes([0.3, 0.85, 0.6, .02])
fig.colorbar(im,cax=cbar_ax,label='Co-Polarized Power (dB)',extend='min',orientation='horizontal') #,ticks=[-5,-,0])
cbar_ax.xaxis.tick_top()
cbar_ax.xaxis.set_label_position("top")
    
plt.savefig('matsuoka_powerloss_suite.png',dpi=300)